In [1]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

# Load from env
load_dotenv()

model = init_chat_model("gpt-4o-mini")

In [ ]:
from pathlib import Path

base_path = Path.cwd().parent.parent.parent

workspace_path = base_path / "agents" / "dev" / "workspace"
print("Workspace Path:", workspace_path)

data_file = workspace_path / "data"
print("Data Path:", data_file)

plot_path = workspace_path / "plot"
print("Plot Path:", plot_path)

Workspace Path: /home/afiq/fyp/fafa-repo/backend/app/agents/dev/workspace
Data Path: /home/afiq/fyp/fafa-repo/backend/app/agents/dev/workspace/data
Plot Path: /home/afiq/fyp/fafa-repo/backend/app/agents/dev/workspace/plot


1. check data given the query
2. req for better data given query (optional)
3. plot
4. return plotting path


In [ ]:
from langchain.tools import tool

example_desc = "Use this tool when you want to answer questions that needs visual information from an image. The image should be a URL or a local file path."
@tool("plotting_tool", description="Use this tool when you need to plot a chart, given that data_exploration_subagent provided enough necessary data in workspaces")
def plotting_tool(data_file: str, instructions: str):
    """
    1. Loads data from data_file.
    2. Writes Matplotlib code based on instructions/schema.
    3. Executes code and captures binary output.
    4. Returns Base64 string.
    """
    import pandas as pd
    import matplotlib.pyplot as plt
    import io
    import base64
    import os

    # 1. Load data from the shared workspace
    file_path = data_path / data_file
    if not os.path.exists(file_path):
        return {"error": f"File {file_path} not found."}
    
    # Support both CSV (from SQLite exploration) and Parquet
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    else:
        df = pd.read_parquet(file_path)

    # 2. Logic to generate code via LLM (Abstracted for this implementation)
    # The 'instructions' and 'schema' are used as context for the LLM
    # In a real tool, this would be a call to your LLM provider.
    
    # 3. Execution (Example of the internal sandbox logic)
    try:
        # We ensure a clean figure state
        plt.clf()
        plt.close('all')
        
        # This is where the LLM generated code would execute via exec()
        # local_vars = {'df': df, 'plt': plt, 'pd': pd}
        # exec(generated_code, {}, local_vars)

        # 4. Capture binary output and return Base64
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        img_str = base64.b64encode(buf.read()).decode('utf-8')
        plt.close('all')
        
        return {
            "plot_b64": img_str,
            "status": "success",
            "caption": f"Visualization generated based on: {instructions}"
        }
    except Exception as e:
        return {"status": "error", "message": str(e)}


/home/afiq/fyp/fafa-repo/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "PlottingTool" shadows an attribute in parent "ValidatedFunction.create_model.<locals>.DecoratorBaseModel"
  warnings.warn(
/home/afiq/fyp/fafa-repo/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "plotting_tool" shadows an attribute in parent "BaseModel"
  warnings.warn(
